<a href="https://colab.research.google.com/github/gopal2812/EPAIAssignment13/blob/main/assignment13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Goal 1
Create a lazy iterator that will return a named tuple of the data in each row. The data types should be appropriate - i.e. if the column is a date, you should be storing dates in the named tuple, if the field is an integer, then it should be stored as an integer, etc.

In [1]:
from collections import namedtuple
from decimal import Decimal
from datetime import datetime

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')
!ls /content/gdrive/MyDrive/nyc*

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/MyDrive/nyc_parking_tickets_extract-1.csv


In [3]:
filename='/content/gdrive/MyDrive/nyc_parking_tickets_extract-1.csv'
open(filename, 'r').readline().strip().split(',')

['Summons Number',
 'Plate ID',
 'Registration State',
 'Plate Type',
 'Issue Date',
 'Violation Code',
 'Vehicle Body Type',
 'Vehicle Make',
 'Violation Description']

In [4]:
# Make a namedtuple with the above keywords as fields in that order
ParkingTickets = namedtuple('ParkingTickets', ('summons_no', 'plate_id', 'regn_state', 'plate_type', 'issue_date', 'violation_code', 'vehicle_type', 'vehicle_make', 'violation_descr'))

In [5]:
# Defining the datatypes in the row
data_types = ['INT', 'STRING', 'STRING', 'STRING', 'DATE', 'INT', 'STRING', 'STRING', 'STRING']

In [6]:
# Function to cast the value to the appropriate datatype
def cast(data_type, value):
    if data_type == 'DOUBLE':
        return float(value)
    elif data_type == 'INT':
        return int(value)
    elif data_type =='DATE':
        return datetime.strptime(value,'%m/%d/%Y').date()
    else:
        return str(value)

In [7]:
#Function which takes one row and sends the elements one by one for casting. Returns a list of casted elements 
def cast_row(data_types, data_row):
    return [cast(data_type, value) 
            for data_type, value in zip(data_types, data_row)]

In [8]:
# Define a function to yield one row at a time (leaving out first line)
# An iterator function (the function becomes an iterator because of the yield in the function) which reads lazily (line by line 
# from the file) and calls the function for casting. Once the elements are casted to proper datatypes, it is returned back.
def get_data(file):
    with open(file, 'r') as f:
      f.readline() # removing first line as it is a header
      yield from (cast_row(data_types, line.strip('\n').split(',')) for line in f)

In [9]:
data_gen = get_data(filename) # data generator

In [10]:
# Goal 1: Lazy iterator
def gen_iters():
    return ParkingTickets(*next(data_gen))

In [11]:
gen_iters() # This is a lazy iterator function going over the data row by row.

ParkingTickets(summons_no=4006478550, plate_id='VAD7274', regn_state='VA', plate_type='PAS', issue_date=datetime.date(2016, 10, 5), violation_code=5, vehicle_type='4D', vehicle_make='BMW', violation_descr='BUS LANE VIOLATION')

#Goal 2
##Calculate the number of violations by car make.

In [12]:
# get list from genrator object
ticket_list = [ParkingTickets(*item) for item in data_gen]
# find the number of elements in list
len(ticket_list)

999

In [13]:
ticket_list[0]

ParkingTickets(summons_no=4006462396, plate_id='22834JK', regn_state='NY', plate_type='COM', issue_date=datetime.date(2016, 9, 30), violation_code=5, vehicle_type='VAN', vehicle_make='CHEVR', violation_descr='BUS LANE VIOLATION')

In [14]:
# Function to count the violations. Uses a dictionary to count the number of violations by each brand car. This would include data 
# even if the name of the car is blank or violation description is blank

ticket_dict = {}

def ctr_dict(vehicle_make)->str:
    def update():
        if not vehicle_make in ticket_dict.keys():
            ticket_dict[vehicle_make] = 0

        ticket_dict[vehicle_make] += 1
        return None

    return update

In [15]:
for item in ticket_list:
    ctr_dict(item.vehicle_make)()

In [16]:
ticket_dict

{'': 5,
 'ACURA': 12,
 'AM/T': 1,
 'AUDI': 12,
 'BMW': 33,
 'BSA': 1,
 'BUICK': 5,
 'CADIL': 9,
 'CHEVR': 76,
 'CHRYS': 12,
 'CITRO': 1,
 'DODGE': 45,
 'FIAT': 1,
 'FIR': 1,
 'FORD': 104,
 'FRUEH': 44,
 'GEO': 1,
 'GMC': 35,
 'GMCQ': 1,
 'HIN': 6,
 'HINO': 2,
 'HONDA': 106,
 'HYUND': 35,
 'INFIN': 13,
 'INTER': 25,
 'ISUZU': 10,
 'JAGUA': 3,
 'JEEP': 22,
 'KENWO': 5,
 'KIA': 8,
 'LEXUS': 26,
 'LINCO': 12,
 'MAZDA': 5,
 'ME/BE': 38,
 'MERCU': 4,
 'MI/F': 1,
 'MINI': 1,
 'MITSU': 11,
 'NISSA': 70,
 'NS/OT': 18,
 'OLDSM': 1,
 'PETER': 1,
 'PLYMO': 1,
 'PONTI': 1,
 'PORSC': 3,
 'ROVER': 5,
 'SAAB': 2,
 'SATUR': 2,
 'SCION': 2,
 'SMART': 3,
 'SPRI': 1,
 'STAR': 1,
 'SUBAR': 18,
 'TOYOT': 112,
 'UD': 1,
 'UPS': 1,
 'UTILI': 1,
 'VOLKS': 8,
 'VOLVO': 12,
 'WORKH': 2,
 'YAMAH': 1}

Get violations By makes

In [17]:
def get_voilations_by_car_make(make_name):
    """Return the number of voilations by a car make"""
    return(ticket_dict[make_name])

In [18]:
get_voilations_by_car_make('BMW')

33